In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
def crawling_news():
    url = 'https://www.newsis.com/realnews/?cid=realnews&day=today&page=1'
    response = requests.get(url)

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.text, 'html.parser')

    # with open('output_file.txt', 'w', encoding='utf-8') as f:
    #  f.write(str(soup))

    # '.txtCont' 셀렉터를 사용하여 각 뉴스 블록 선택
    news_blocks = soup.select('.txtCont')

    # 각 뉴스 블록 내에서 필요한 정보 추출
    for block in news_blocks:
        if block.select_one('.tit a'):
            # 'tit a' 태그의 텍스트(제목)와 'href'(링크) 추출
            title_link = block.select_one('.tit a')
            description = block.select_one('.txt a')
            description_txt = description.text.strip()
            title = title_link.text.strip()
            link = title_link['href']
            # 'time' 클래스의 텍스트(시간 정보) 추출
            reporter = block.select_one('.time').span.text.strip()
            time = block.select_one('.time')
            if time.span:
                time.span.decompose()
            
            time = time.text.strip()

            print(f'Title: {title}\nDescription: {description_txt}\nLink: {link}\nTime: {time}\nReporter: {reporter}\n')
        else: 
            break

In [6]:
crawling_news()

Title: "NO 왁싱으로 간다"…김종국·양세찬, '팬츠리스룩' 도전?
Description: SBS TV 예능물 '런닝맨' 멤버들이 '팬츠리스룩'을 건 '힙 전쟁'을 펼친다.

31일 오후 6시10분 방송하는 '런닝맨'은 '힙이 차오른다' 레이스로 꾸며진다. 성수동 핫플레이스부터 패션 컨설팅까지 MZ들에게 유행하는 힙을 찾아 떠났다. 

하지만 장소에서 발생한 트렌드 따라잡기 비용은 멤버들끼리 나눠 내야 했다. 남은 금액에 따라 벌칙이 좌지
Link: /view/?id=NISX20240331_0002681654
Time: 2024.03.31 03:09:00
Reporter: 이재훈기자

Title: '결승골' 손흥민, 루턴전 평점 8.4…양 팀 최고점
Description: 잉글랜드 프로축구 프리미어리그(EPL) 토트넘홋스퍼 '주장' 손흥민이 역전 결승골을 기록한 활약을 인정받아 팀 내 최고 평점을 받았다.

토트넘은 31일 0시(한국시각) 영국 런던의 토트넘홋스퍼 스타디움에서 열린 '2023~2024 EPL' 30라운드 루턴 타운전에서 2-1 역전승을 거뒀다.

이로써 토트넘은 애스턴 빌라를 제치고 4위가 됐다.
Link: /view/?id=NISX20240330_0002681651
Time: 2024.03.31 02:21:12
Reporter: 김진엽기자

Title: '손흥민 결승골' 토트넘, 루턴에 2-1 역전승…4위 등극
Description: 잉글랜드 프로축구 프리미어리그(EPL) 토트넘홋스퍼 '주장' 손흥민이 역전 결승골을 기록하며 팀에 승점 3을 안겼다.

토트넘은 31일 0시(한국시각) 영국 런던의 토트넘홋스퍼 스타디움에서 열린 '2023~2024 EPL' 30라운드 루턴 타운전에서 2-1 역전승을 거뒀다.

이로써 17승5무7패(승점 56)를 기록한 토트넘은 애스턴 빌라를 밀어
Link: /view/?id=NISX20240330_0002681650
Time: 2024.03.31 02:00:22
Reporter: 김진엽기자

Title: 필